<a href="https://colab.research.google.com/github/Marco-barthem/email-spam-classification/blob/main/spam_email_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Teste

In [6]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Marco-barthem/email-spam-classification/refs/heads/main/Spam%20Email%20raw%20text%20for%20NLP.csv')
df

,CATEGORY,MESSAGE,FILE_NAME
0,1,"Dear Homeowner,\n\n \n\nInterest Rates are at ...",00249.5f45607c1bffe89f60ba1ec9f878039a
1,1,ATTENTION: This is a MUST for ALL Computer Use...,00373.ebe8670ac56b04125c25100a36ab0510
2,1,This is a multi-part message in MIME format.\n...,00214.1367039e50dc6b7adb0f2aa8aba83216
3,1,IMPORTANT INFORMATION:\n\n\n\nThe new domain n...,00210.050ffd105bd4e006771ee63cabc59978
4,1,This is the bottom line. If you can GIVE AWAY...,00033.9babb58d9298daa2963d4f514193d7d6
...,...,...,...
5791,0,"I'm one of the 30,000 but it's not working ver...",00609.dd49926ce94a1ea328cce9b62825bc97
5792,0,Damien Morton quoted:\n\n>W3C approves HTML 4 ...,00957.e0b56b117f3ec5f85e432a9d2a47801f
5793,0,"On Mon, 2002-07-22 at 06:50, che wrote:\n\n\n\...",01127.841233b48eceb74a825417d8d918abf8
5794,0,"Once upon a time, Manfred wrote :\n\n\n\n> I w...",01178.5c977dff972cd6eef64d4173b90307f0


In [7]:
df['CATEGORY'].value_counts()

,count
CATEGORY,
0,3900
1,1896


In [10]:
import nltk #Natural Language Tool Kit
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [22]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import ToktokTokenizer
tokenizer = ToktokTokenizer()
lemmatizer = WordNetLemmatizer()
stopwords = stopwords.words('english')

In [25]:
frase_teste = ['Hi im just a test cars']
teste = tokenizer.tokenize(frase_teste[0])
teste
lemantize = [lemmatizer.lemmatize(i) for i in teste]
lemantize
lemantize_stop = [t for t in lemantize if t  not in stopwords]
lemantize_stop

['Hi', 'im', 'test', 'car']

In [39]:
def message_to_token_list(s):
    tokens = tokenizer.tokenize(s)
    lower_case = [t.lower() for t in tokens if t.isalpha()]  # filtra símbolos e acentos
    lemmatizer_words = [lemmatizer.lemmatize(i) for i in lower_case]
    useful_tokens = [u for u in lemmatizer_words if u not in stopwords]
    return useful_tokens

In [40]:
df = df.sample(frac=1,random_state=1)
df=df.reset_index(drop=True)

split_index = int(len(df)*0.8)
train_df,test_df = df[:split_index],df[split_index:]

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df,test_df

(      CATEGORY                                            MESSAGE  \
 0            0  I think that this and other articles confuse S...   
 1            1  This is a multi-part message in MIME format.\n...   
 2            1  <HR>\n\n<html>\n\n<div bgcolor=3D"#FFFFCC">\n\...   
 3            0  http://www.guardian.co.uk/religion/Story/0,276...   
 4            1  Lower your mortgage rate and/or payment! FREE ...   
 ...        ...                                                ...   
 4631         0  I've picked up these patches and will submit t...   
 4632         0  Anyone interested in a pgp/gpg key-signing get...   
 4633         1  This is a multi-part message in MIME format.\n...   
 4634         0  All,\n\n\n\nIs it just me or has there been a ...   
 4635         0  URL: http://www.aaronsw.com/weblog/000609\n\nD...   
 
                                    FILE_NAME  
 0     00318.5b1d22882c6fbfebcebe12a792b3c6c8  
 1     01384.e23f94030a4393f0825eacd9de99eb31  
 2     00707.c

In [41]:
token_counter = {}

for message in train_df['MESSAGE']:
  message_as_tkn_lst = message_to_token_list(message)
  for t in message_as_tkn_lst:
    if t not in token_counter:
      token_counter[t] = 1
    else:
      token_counter[t] += 1

len(token_counter)

30415

In [51]:
token_counter

{'think': 1022,
 'article': 191,
 'confuse': 18,
 'socialism': 43,
 'libertarianism': 7,
 'implemented': 33,
 'north': 129,
 'america': 295,
 'exactly': 189,
 'shining': 6,
 'pinnacle': 5,
 'economic': 176,
 'try': 490,
 'starting': 165,
 'telephone': 112,
 'company': 1232,
 'u': 1459,
 'even': 1126,
 'worse': 52,
 'take': 984,
 'year': 1263,
 'get': 2846,
 'blessing': 12,
 'permit': 35,
 'raj': 11,
 'fcc': 23,
 'puc': 5,
 'ptts': 5,
 'decidedly': 11,
 'socialist': 30,
 'leaning': 5,
 'canada': 96,
 'industry': 200,
 'crtc': 5,
 'yet': 292,
 'despite': 70,
 'intense': 15,
 'regulation': 79,
 'paper': 156,
 'pushing': 15,
 'well': 762,
 'regulatory': 29,
 'scrutiny': 15,
 'ftc': 6,
 'sec': 19,
 'irs': 26,
 'executive': 106,
 'telecom': 77,
 'managed': 53,
 'bilk': 6,
 'investment': 316,
 'community': 186,
 'look': 736,
 'ten': 92,
 'billion': 173,
 'finished': 43,
 'routine': 42,
 'quadruple': 12,
 'lutz': 7,
 'laying': 12,
 'hundred': 266,
 'thousand': 337,
 'worker': 42,
 'came': 197,

In [43]:
def keep_token(p_token,threshold ):
  if p_token not in token_counter:
    return False
  else:
    return token_counter[p_token] >= threshold


In [46]:
features=set()

for i in token_counter:
  if keep_token(i,2000):
    features.add(i)

features =list(features)
features #bag of words

['helvetica',
 'arial',
 'email',
 'wa',
 'would',
 'ha',
 'get',
 'verdana',
 'people',
 'one',
 'center',
 'free',
 'list',
 'time',
 'new']

In [47]:
token_to_index_mapping = {t:i for t,i in zip(features,range(len(features)))} #dicionário que mapeia cada token  para um índice numérico.
token_to_index_mapping


{'helvetica': 0,
 'arial': 1,
 'email': 2,
 'wa': 3,
 'would': 4,
 'ha': 5,
 'get': 6,
 'verdana': 7,
 'people': 8,
 'one': 9,
 'center': 10,
 'free': 11,
 'list': 12,
 'time': 13,
 'new': 14}

In [ ]:
import numpy as np
def message_to_count_vector(message):
  count_vector = np.zeros(len(features))

